# Results

In [ ]:
# ruff: noqa: E722
import json
import random
import re
from pathlib import Path
from time import perf_counter

import cc3d
import fill_voids
import itk
import nibabel as nib
import numpy as np
import trimesh
from numpy.typing import NDArray
from tqdm import tqdm

from edge_mender import EdgeMender, MeshGenerator

itk.Image  # pyright: ignore[reportAttributeAccessIssue]  # noqa: B018

import itk.CuberillePython  # noqa: E402

# Prep paths
results_dir = Path("results")
results_dir.mkdir(exist_ok=True)
mesh_dir = results_dir / "meshes"
mesh_dir.mkdir(exist_ok=True)


# Load previous results
results_path = results_dir / "results.json"
if results_path.exists():
    with results_path.open() as f:
        results = json.load(f)
else:
    results = {}

# NOTE: https://github.com/InsightSoftwareConsortium/ITKCuberille/issues/90
cuberille_kernel_crash = [
    "Task05_Prostate",
    "pancreas_08",
    "pancreas_12",
    "pancreas_40",
    "pancreas_41",
    "Task08_HepaticVessel",
    "spleen_2",
    "colon_069",
]


def mesh_is_malformed(mesh: trimesh.Trimesh) -> bool:
    """Check if a mesh is malformed."""
    try:
        EdgeMender(mesh).validate(spacing=(1.0, 1.0, 1.0))
    except ValueError:
        return True
    return False


# Loop through files
paths = [
    path
    for path in Path("data").glob("*/labelsTr/*.nii.gz")
    if not path.name.startswith("._")  # Skip macOS metadata files
    # and np.random.rand() < 0.1
]
random.shuffle(paths)
pbar = tqdm(paths, smoothing=0.1)
for path in pbar:
    pbar.set_description(str(path))
    name = re.sub(r".nii$", "", path.stem, count=1)
    # Skip if already processed
    if name in results:
        continue

    result = {
        "path": str(path),
        "cuberille": {},
        "surface_nets": {},
        "nmes": {},
        "volume": {},
        "repair": {},
    }

    # Load data
    img = nib.load(path)  # pyright: ignore[reportPrivateImportUsage]
    data: NDArray = img.get_fdata()  # pyright: ignore[reportAttributeAccessIssue]

    result["size"] = data.size
    result["shape"] = list(data.shape)
    result["voxel_count"] = np.count_nonzero(data).item()

    # Mesh with Surface Nets
    try:
        start = perf_counter()
        surface_nets_mesh = MeshGenerator.to_mesh_surface_nets(data)
        result["surface_nets"]["time"] = perf_counter() - start
        result["surface_nets"]["malformed"] = mesh_is_malformed(surface_nets_mesh)
    except KeyboardInterrupt:
        break
    except:
        result["surface_nets"]["failed"] = True

    # If the mesh crashes the program, use Surface Nets instead
    if any(x in str(path) for x in cuberille_kernel_crash):
        surface_nets_mesh.export(mesh_dir / f"{name}.stl")
        data = cc3d.largest_k(data, k=1, connectivity=6).astype(np.uint8)
        data = fill_voids.fill(data, in_place=True)
        try:
            surface_nets_mesh = MeshGenerator.to_mesh_surface_nets(data)
        except KeyboardInterrupt:
            break
        except:
            result["surface_nets"]["repaired"] = False
        else:
            surface_nets_mesh.export(mesh_dir / f"{name}_repaired.stl")
            result["surface_nets"]["repaired"] = not mesh_is_malformed(
                surface_nets_mesh,
            )
        if not result["surface_nets"]["repaired"]:
            continue
        mesh = surface_nets_mesh
    else:
        # Mesh with Cuberille
        try:
            start = perf_counter()
            cuberille_mesh = MeshGenerator.to_mesh_cuberille(data)
            result["cuberille"]["time"] = perf_counter() - start
            cuberille_mesh.export(mesh_dir / f"{name}.stl")
            result["cuberille"]["malformed"] = mesh_is_malformed(cuberille_mesh)
        except KeyboardInterrupt:
            break
        except:
            result["cuberille"]["failed"] = True
            continue

        # Attempt to fix malformed meshes
        if result["cuberille"]["malformed"]:
            data = cc3d.largest_k(data, k=1, connectivity=6).astype(np.uint8)
            try:
                cuberille_mesh = MeshGenerator.to_mesh_cuberille(data)
            except KeyboardInterrupt:
                break
            except:
                result["cuberille"]["repaired"] = False
            else:
                cuberille_mesh.export(mesh_dir / f"{name}_repaired.stl")
                result["cuberille"]["repaired"] = not mesh_is_malformed(cuberille_mesh)
            if not result["cuberille"]["repaired"]:
                continue
        mesh = cuberille_mesh

    # Measure properties before
    result["nmes"]["before"] = len(EdgeMender(mesh).find_non_manifold_edges()[2])
    result["volume"]["before"] = mesh.volume

    # Repair mesh
    try:
        mesh._cache.clear()  # noqa: SLF001
        start = perf_counter()
        EdgeMender(mesh).repair()
        result["repair"]["time"] = perf_counter() - start
        result["repair"]["failed"] = False
    except KeyboardInterrupt:
        break
    except:
        result["repair"]["failed"] = True

    # Measure properties after
    result["nmes"]["after"] = len(EdgeMender(mesh).find_non_manifold_edges()[2])
    result["volume"]["after"] = mesh.volume
    result["is_watertight"] = mesh.is_watertight

    # Save results
    results[name] = result
    with results_path.open("w") as f:
        json.dump(results, f, indent=4)

In [ ]:
# TODO: Add file size

# Statistics and Plots

In [ ]:
import json
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Load previous results
results_dir = Path("results")
plots_dir = results_dir / "plots"
plots_dir.mkdir(exist_ok=True)
results_path = results_dir / "results.json"
if results_path.exists():
    with results_path.open() as f:
        results = json.load(f)
paths = [
    path
    for path in Path("data").glob("*/labelsTr/*.nii.gz")
    if not path.name.startswith("._")  # Skip macOS metadata files
]

generation_summary = pd.DataFrame(
    {
        "Total": [
            len(paths),
            len(paths),
        ],
        "Meshed": [
            sum(1 for v in results.values() if "time" in v["cuberille"]),
            sum(1 for v in results.values() if "time" in v["surface_nets"]),
        ],
        "Average Size": [
            np.mean([v["size"] for v in results.values()]),
            np.mean([v["size"] for v in results.values()]),
        ],
        "Average Voxel Count": [
            np.mean([v["voxel_count"] for v in results.values()]),
            np.mean([v["voxel_count"] for v in results.values()]),
        ],
        "Average Time (s)": [
            np.mean(
                [
                    v["cuberille"]["time"]
                    for v in results.values()
                    if "time" in v["cuberille"]
                ],
            ),
            np.mean(
                [
                    v["surface_nets"]["time"]
                    for v in results.values()
                    if "time" in v["surface_nets"]
                ],
            ),
        ],
        "# Failed": [
            sum(1 for v in results.values() if v["cuberille"].get("failed")),
            sum(1 for v in results.values() if v["surface_nets"].get("failed")),
        ],
        "# Malformed": [
            sum(1 for v in results.values() if v["cuberille"].get("malformed")),
            sum(1 for v in results.values() if v["surface_nets"].get("malformed")),
        ],
        "# Repaired": [
            sum(1 for v in results.values() if v["cuberille"].get("repaired")),
            sum(1 for v in results.values() if v["surface_nets"].get("repaired")),
        ],
    },
    index=["Cuberille", "Surface Nets"],
)

repair_count = len([v for v in results.values() if "before" in v["nmes"]])
repair_summary = pd.DataFrame(
    {
        "values": [
            repair_count,
            np.mean(
                [
                    v["nmes"]["before"]
                    for v in results.values()
                    if "before" in v["nmes"]
                ],
            ),
            np.mean(
                [v["nmes"]["after"] for v in results.values() if "after" in v["nmes"]],
            ),
            np.mean(
                [
                    v["volume"]["before"]
                    for v in results.values()
                    if "before" in v["volume"]
                ],
            ),
            np.mean(
                [
                    v["volume"]["after"] - v["volume"]["before"]
                    for v in results.values()
                    if "before" in v["volume"] and "after" in v["volume"]
                ],
            ),
            np.mean(
                [
                    v["repair"]["time"]
                    for v in results.values()
                    if "time" in v["repair"]
                ],
            ),
            sum(
                1
                for v in results.values()
                if not v["repair"]["failed"]
                and v["nmes"]["after"] == 0
                and v["is_watertight"]
            )
            / repair_count,
        ],
    },
    index=[
        "Count",
        "Average # of NMEs Before",
        "Average # of NMEs After",
        "Average Volume",
        "Average Volume Change",
        "Average Repair Time (s)",
        "Success Rate",
    ],
)

with pd.ExcelWriter(results_dir / "results.xlsx", engine="openpyxl") as writer:
    generation_summary.to_excel(writer, sheet_name="Mesh Generation")
    repair_summary.to_excel(writer, sheet_name="Repair", header=False)

In [ ]:
# TODO: Update to include min, max, and standard deviation
# TODO: Add repair statistics excluding meshes with no NMEs
# TODO: Add time per repair statistics
# TODO: Add relative repair time vs. mesh generation time statistics

In [ ]:
# Sizes
sizes = [v["size"] for v in results.values()]

plt.hist(sizes, bins=20, edgecolor="black")
plt.xlabel("Size (voxels)")
plt.ylabel("Frequency")
plt.grid(alpha=0.2)
ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.savefig(plots_dir / "size_histogram.png")
plt.show()

In [ ]:
# Size vs. Mesh Time per Method
cuberille_sizes = [v["size"] for v in results.values() if "time" in v["cuberille"]]
cuberille_times = [
    v["cuberille"]["time"] for v in results.values() if "time" in v["cuberille"]
]
surface_nets_sizes = [
    v["size"] for v in results.values() if "time" in v["surface_nets"]
]
surface_nets_times = [
    v["surface_nets"]["time"] for v in results.values() if "time" in v["surface_nets"]
]

plt.scatter(cuberille_sizes, cuberille_times, label="Cuberille")
plt.scatter(surface_nets_sizes, surface_nets_times, label="Surface Nets")
plt.xlabel("Size (voxels)")
plt.ylabel("Time (seconds)")
plt.legend()
plt.grid(alpha=0.2)
ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.savefig(plots_dir / "size_vs_mesh_time.png")
plt.show()

In [ ]:
# Voxel Count vs. Mesh Time per Method
cuberille_voxel_counts = [
    v["voxel_count"] for v in results.values() if "time" in v["cuberille"]
]
cuberille_times = [
    v["cuberille"]["time"] for v in results.values() if "time" in v["cuberille"]
]
surface_nets_voxel_counts = [
    v["voxel_count"] for v in results.values() if "time" in v["surface_nets"]
]
surface_nets_times = [
    v["surface_nets"]["time"] for v in results.values() if "time" in v["surface_nets"]
]

plt.scatter(cuberille_voxel_counts, cuberille_times, label="Cuberille")
plt.scatter(surface_nets_voxel_counts, surface_nets_times, label="Surface Nets")
plt.xlabel("Foreground Voxels")
plt.ylabel("Time (seconds)")
plt.legend()
plt.grid(alpha=0.2)
ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.savefig(plots_dir / "voxel_count_vs_mesh_time.png")
plt.show()

In [ ]:
# Size vs. Non-manifold Edges
sizes = [v["size"] for v in results.values() if "before" in v["nmes"]]
nmes = [v["nmes"]["before"] for v in results.values() if "before" in v["nmes"]]

plt.scatter(sizes, nmes)
plt.xlabel("Size (voxels)")
plt.ylabel("Non-Manifold Edges")
plt.grid(alpha=0.2)
ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.savefig(plots_dir / "size_vs_nmes.png")
plt.show()

In [ ]:
# Voxel Count vs. Non-Manifold Edges
voxel_counts = [v["voxel_count"] for v in results.values() if "before" in v["nmes"]]
nmes = [v["nmes"]["before"] for v in results.values() if "before" in v["nmes"]]

plt.scatter(voxel_counts, nmes)
plt.xlabel("Foreground Voxels")
plt.ylabel("Non-Manifold Edges")
plt.grid(alpha=0.2)
ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.savefig(plots_dir / "voxel_counts_vs_nmes.png")
plt.show()

In [ ]:
# Size vs. Repair Time
sizes = [
    v["size"]
    for v in results.values()
    if v["nmes"]["before"] != 0 and "time" in v["repair"]
]
times = [
    v["repair"]["time"]
    for v in results.values()
    if v["nmes"]["before"] != 0 and "time" in v["repair"]
]

plt.scatter(sizes, times)
plt.xlabel("Size (voxels)")
plt.ylabel("Time (seconds)")
plt.grid(alpha=0.2)
ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.savefig(plots_dir / "size_vs_repair_time.png")
plt.show()

In [ ]:
# Voxel Count vs. Repair Time
voxel_counts = [
    v["voxel_count"]
    for v in results.values()
    if v["nmes"]["before"] != 0 and "time" in v["repair"]
]
times = [
    v["repair"]["time"]
    for v in results.values()
    if v["nmes"]["before"] != 0 and "time" in v["repair"]
]

plt.scatter(voxel_counts, times)
plt.xlabel("Foreground Voxels")
plt.ylabel("Time (seconds)")
plt.grid(alpha=0.2)
ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.savefig(plots_dir / "voxel_count_vs_repair_time.png")
plt.show()

In [ ]:
# Non-Manifold Edges vs. Repair Time
nmes = [
    v["nmes"]["before"]
    for v in results.values()
    if v["nmes"]["before"] != 0 and "time" in v["repair"]
]
times = [
    v["repair"]["time"]
    for v in results.values()
    if v["nmes"]["before"] != 0 and "time" in v["repair"]
]

plt.scatter(nmes, times)
plt.xlabel("Non-Manifold Edges")
plt.ylabel("Time (seconds)")
plt.grid(alpha=0.2)
ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.savefig(plots_dir / "nmes_vs_repair_time.png")
plt.show()

In [ ]:
# Size vs. Time per Repair
sizes = [
    v["size"]
    for v in results.values()
    if v["nmes"]["before"] != 0 and "time" in v["repair"]
]
times = [
    v["repair"]["time"] / v["nmes"]["before"]
    for v in results.values()
    if v["nmes"]["before"] != 0 and "time" in v["repair"]
]

plt.scatter(sizes, times)
plt.xlabel("Size (voxels)")
plt.ylabel("Time (seconds)")
plt.grid(alpha=0.2)
ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.savefig(plots_dir / "size_vs_time_per_repair.png")
plt.show()

In [ ]:
# Voxel Count vs. Time per Repair
voxel_counts = [
    v["voxel_count"]
    for v in results.values()
    if v["nmes"]["before"] != 0 and "time" in v["repair"]
]
times = [
    v["repair"]["time"] / v["nmes"]["before"]
    for v in results.values()
    if v["nmes"]["before"] != 0 and "time" in v["repair"]
]

plt.scatter(voxel_counts, times)
plt.xlabel("Foreground Voxels")
plt.ylabel("Time (seconds)")
plt.grid(alpha=0.2)
ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.savefig(plots_dir / "voxel_count_vs_time_per_repair.png")
plt.show()

In [ ]:
# Non-Manifold Edges vs. Time per Repair
nmes = [
    v["nmes"]["before"]
    for v in results.values()
    if v["nmes"]["before"] != 0 and "time" in v["repair"]
]
times = [
    v["repair"]["time"] / v["nmes"]["before"]
    for v in results.values()
    if v["nmes"]["before"] != 0 and "time" in v["repair"]
]

plt.scatter(nmes, times)
plt.xlabel("Non-Manifold Edges")
plt.ylabel("Time (seconds)")
plt.grid(alpha=0.2)
ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.savefig(plots_dir / "nmes_vs_time_per_repair.png")
plt.show()

In [ ]:
# Mesh Time per Method vs. Repair Time
cuberille_mesh_times = [
    v["cuberille"]["time"]
    for v in results.values()
    if v["nmes"]["before"] != 0 and "time" in v["repair"] and "time" in v["cuberille"]
]
cuberille_repair_times = [
    v["repair"]["time"]
    for v in results.values()
    if v["nmes"]["before"] != 0 and "time" in v["repair"] and "time" in v["cuberille"]
]
surface_nets_mesh_times = [
    v["surface_nets"]["time"]
    for v in results.values()
    if v["nmes"]["before"] != 0
    and "time" in v["repair"]
    and "time" in v["surface_nets"]
]
surface_nets_repair_times = [
    v["repair"]["time"]
    for v in results.values()
    if v["nmes"]["before"] != 0
    and "time" in v["repair"]
    and "time" in v["surface_nets"]
]

plt.scatter(cuberille_mesh_times, cuberille_repair_times, label="Cuberille")
plt.scatter(surface_nets_mesh_times, surface_nets_repair_times, label="Surface Nets")
plt.xlabel("Mesh Time (seconds)")
plt.ylabel("Repair Time (seconds)")
plt.legend()
plt.grid(alpha=0.2)
ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.set_aspect("equal", adjustable="datalim")
plt.savefig(plots_dir / "mesh_time_vs_repair_time.png")
plt.show()